In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = 

In [4]:
! pip install langchain chromadb openai tiktoken pypdf langchain_openai langchain-community

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma


/home/shreyash-ubuntu/langchain_tut/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [4]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [5]:
model = GoogleGenerativeAIEmbeddings(model= "models/embedding-001")
vector_store = Chroma(
    embedding_function= model,
    persist_directory='my_chroma_db',
    collection_name='sample'
)

/tmp/ipykernel_10252/2110276425.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [6]:
# add documents
vector_store.add_documents(docs)

['42664f11-4add-400a-88dc-39c3c5d26679',
 '661467c8-89da-4a22-a098-3263d441fff8',
 '896a4402-83d8-44b1-8e00-49a25521e7a8',
 '986f9cbf-1cd9-4e1f-a9d6-17271229cd85',
 'fabcae33-2526-4ba6-a572-bc9890bdc5dd']

In [7]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['42664f11-4add-400a-88dc-39c3c5d26679',
  '661467c8-89da-4a22-a098-3263d441fff8',
  '896a4402-83d8-44b1-8e00-49a25521e7a8',
  '986f9cbf-1cd9-4e1f-a9d6-17271229cd85',
  'fabcae33-2526-4ba6-a572-bc9890bdc5dd'],
 'embeddings': array([[ 1.35614481e-02, -7.62679800e-03, -2.51727290e-02, ...,
          1.35761239e-02,  3.51136659e-05, -7.35754846e-03],
        [ 3.18830460e-02, -3.36795598e-02, -4.39120680e-02, ...,
          1.08157219e-02, -1.05898809e-02, -1.52005616e-03],
        [ 1.58164389e-02,  3.38992523e-03, -2.23509353e-02, ...,
          1.95286470e-03, -7.18898326e-02, -8.88233818e-03],
        [ 1.01624681e-02, -5.16264960e-02, -2.89599448e-02, ...,
          3.29840817e-02, -2.09922232e-02,  1.78275332e-02],
        [-8.94617348e-04,  1.97164640e-02, -9.64438636e-03, ...,
          2.34126463e-03, -1.33365402e-02, -1.25717660e-02]],
       shape=(5, 768)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his ag

In [8]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.')]

In [9]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.770696222782135),
 (Document(metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.'),
  0.8030722737312317)]

In [10]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.871087908744812),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.8778100609779358)]

In [13]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='42664f11-4add-400a-88dc-39c3c5d26679', document=updated_doc1)


In [14]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['42664f11-4add-400a-88dc-39c3c5d26679',
  '661467c8-89da-4a22-a098-3263d441fff8',
  '896a4402-83d8-44b1-8e00-49a25521e7a8',
  '986f9cbf-1cd9-4e1f-a9d6-17271229cd85',
  'fabcae33-2526-4ba6-a572-bc9890bdc5dd'],
 'embeddings': array([[ 0.02001663, -0.01274028, -0.02455907, ...,  0.00292782,
         -0.01331383,  0.00581167],
        [ 0.03188305, -0.03367956, -0.04391207, ...,  0.01081572,
         -0.01058988, -0.00152006],
        [ 0.01581644,  0.00338993, -0.02235094, ...,  0.00195286,
         -0.07188983, -0.00888234],
        [ 0.01016247, -0.0516265 , -0.02895994, ...,  0.03298408,
         -0.02099222,  0.01782753],
        [-0.00089462,  0.01971646, -0.00964439, ...,  0.00234126,
         -0.01333654, -0.01257177]], shape=(5, 768)),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple ce

In [15]:
# delete document
vector_store.delete(ids=['42664f11-4add-400a-88dc-39c3c5d26679'])

In [16]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['661467c8-89da-4a22-a098-3263d441fff8',
  '896a4402-83d8-44b1-8e00-49a25521e7a8',
  '986f9cbf-1cd9-4e1f-a9d6-17271229cd85',
  'fabcae33-2526-4ba6-a572-bc9890bdc5dd'],
 'embeddings': array([[ 0.03188305, -0.03367956, -0.04391207, ...,  0.01081572,
         -0.01058988, -0.00152006],
        [ 0.01581644,  0.00338993, -0.02235094, ...,  0.00195286,
         -0.07188983, -0.00888234],
        [ 0.01016247, -0.0516265 , -0.02895994, ...,  0.03298408,
         -0.02099222,  0.01782753],
        [-0.00089462,  0.01971646, -0.00964439, ...,  0.00234126,
         -0.01333654, -0.01257177]], shape=(4, 768)),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  'MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.',
  'Jasprit Bumrah is